In [1]:
from syft import PhiTensor as PT

In [2]:
from syft.core.adp.data_subject_list import DataSubject
import numpy as np

ishan = np.array([DataSubject(["Ishan"])])

In [3]:
from syft import lazyrepeatarray as lra

upper_bound = lra(data=5, shape=(3,3))
lower_bound = lra(data=-5, shape=(3,3))

reference_data = np.random.randint(-5, 5, (3,3))

In [4]:
reference_tensor = PT(
    child=np.array(reference_data),
    data_subject=ishan,
    max_vals=upper_bound,
    min_vals=lower_bound,
)
gamma_tensor = reference_tensor.gamma

aux_tensor = gamma_tensor == reference_data
result = aux_tensor.any()
assert result.child
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(axis=0)
assert result.shape == (reference_data.shape[0],)
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(keepdims=True)
assert result.shape == (1, 1)
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

result = (gamma_tensor == reference_data).any(keepdims=True, axis=0)
assert result.shape == (1, reference_tensor.shape[0])
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
assert (result.func(state) == result.child).all()

condition = list(
    np.random.choice(a=[False, True], size=(reference_data.shape[0] - 1))
)
condition.append(
    True
)  # If condition = [False, False, False ... False], this test will fail
result = (gamma_tensor == reference_data).any(where=condition)
assert result.child
assert list(result.sources.keys()) == [reference_tensor.id]
state = {}
for key in result.sources:
    state[key] = result.sources[key].child
# print(state)
# print(result.func(state))

In [7]:
state

{<UID: d8c5a78f28e14c8393ac8e67b378fc5d>: array([[-1,  2, -1],
        [-2,  4,  2],
        [-1, -2,  2]])}

In [32]:
func = lambda state: list(jnp.any(list(state.values())[0], axis=None, keepdims=False, where=condition))

In [33]:
list(state.values())[0]

array([[-1,  2, -1],
       [-2,  4,  2],
       [-1, -2,  2]])

In [39]:
condition

[False, True, True]

In [41]:
np.any(list(state.values())[0], where=condition)

True

In [42]:
help(jnp.any)

Help on function any in module jax._src.numpy.reductions:

any(a, axis: Union[int, Tuple[int, ...], NoneType] = None, out=None, keepdims=None, *, where=None)
    Test whether any array element along a given axis evaluates to True.
    
    LAX-backend implementation of :func:`numpy.any`.
    
    *Original docstring below.*
    
    Returns single boolean unless `axis` is not ``None``
    
    Parameters
    ----------
    a : array_like
        Input array or object that can be converted to an array.
    axis : None or int or tuple of ints, optional
        Axis or axes along which a logical OR reduction is performed.
        The default (``axis=None``) is to perform a logical OR over all
        the dimensions of the input array. `axis` may be negative, in
        which case it counts from the last to the first axis.
    keepdims : bool, optional
        If this is set to True, the axes which are reduced are left
        in the result as dimensions with size one. With this option,
  

In [45]:
jnp.any(list(state.values())[0], axis=None, keepdims=False, where=np.array(condition))

DeviceArray(True, dtype=bool)

In [34]:
func(state)

TypeError: Field elements must be 2- or 3-tuples, got 'Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=0/2)>'

In [15]:
result.reconstruct()

TypeError: reconstruct() missing 1 required positional argument: 'state'

In [10]:
result.child

array(True)

In [11]:
result.func({})

KeyError: <UID: d8c5a78f28e14c8393ac8e67b378fc5d>

In [9]:
result.func(state)

TypeError: Field elements must be 2- or 3-tuples, got 'Traced<ShapedArray(bool[])>with<DynamicJaxprTrace(level=0/2)>'

In [5]:
assert result.func(state).shape == result.child.shape
assert (result.func(state) == result.child).all()  # TODO 0.7: debug this